### This code which rus on python 3 accompanies the paper "A survey of  testicular texture on canine ultrasound images" Submitted to Frontiers in Veterinary Science" Panida Pongvittayanon et al.2023.

If this juyter notebook is run on a local machine, the notebook file "texture.ipynb"  and the directory "images", containing  one or more images files (in "png" format) should be present in the same directory.  The code will generate an output file called "output.csv" containg the texture anaysis results and a "png" image file which shows the ROI selected data.

The cell below imports the python modules required for the texture analysis.

In [ ]:
import cv2
import csv
from csv import writer
import os
import skimage
import skimage.measure


from skimage.io import imread
from skimage import io
from skimage.feature import greycomatrix
from skimage.feature import greycoprops

from skimage.transform import integral_image


The cell below checks if an output file "output.csv"  exists, if it does data from the images analysis of will be appended to it. Ifthe file does not exist , it is created as a blank csv file with headers.

In [ ]:
home = os.getcwd()

if not os.path.exists(home+'/output.csv'):
    print('No output file exists, it will be created and called output.csv')
    f = open('output.csv', 'w')
  
    # create the csv writer
    writer_head = csv.writer(f)
    header=['file_name','entropy','ASM','contrast','energy','dissimilarity','correlation','homogeneity','roi_1','roi_3','roi_0','roi_2','dynamic_range','depth','nodule_present','mineralization_present','comment']
    # write the header
    writer_head.writerow(header)

    # close the file
    f.close()


The cell below defines a function "crop_save_texture" which opens an image, creates an interface for selecting a region of interest (ROI), runs the texture analysis on that ROI, saves the ROI as an image and save the texture and user input data to a results file called "output.csv".

In [ ]:
def crop_save_texture(image, image_name):
    roi = cv2.selectROI(image)
    print("Use the mouse to select and ROI, then press enter or return to save the ROI , or press 'c' to cancel.")
    #Crop selected roi from raw image
    roi_cropped = image[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]

    # Save the cropped image
    cv2.imwrite((str(image_name)[0:-4]+'_crop'+'.png'), roi_cropped)

    glcm = greycomatrix(roi_cropped, [5], [0], 256, symmetric=True, normed=True)
    ts = []
    us = []
    vs = []
    ws = []
    xs = []
    ys = []
    zs = []
    ts.append(skimage.measure.shannon_entropy(roi_cropped))
    us.append(greycoprops(glcm, 'ASM')[0, 0])    
    vs.append(greycoprops(glcm, 'contrast')[0, 0])
    ws.append(greycoprops(glcm, 'energy')[0, 0])    
    xs.append(greycoprops(glcm, 'dissimilarity')[0, 0])
    ys.append(greycoprops(glcm, 'correlation')[0, 0])
    zs.append(greycoprops(glcm, 'homogeneity')[0, 0])

    feature_type = ['type-2-x', 'type-2-y']
    feature_coord= None

    dr =input("Enter a value for dynamic range: ")
    depth = input("Enter a value image depth: ")
    nodule = input("Enter a value for nodule 1 present zero not present: ")
    mineral = input("Enter a value for mineralization 2 present, zero not present: ")
    comment = input("Enter an optional free text comment: ")

    print("entrophy is {}\n\
    ASM is {}\n\
    Contrast is {}\n\
    Energy is {}\n\
    Dissimilarity is {}\n\
    Correlation is {}\n\
    Homogeneity is {}\n"\
    .format(ts,us,vs,ws,xs,ys,zs))
 
    # List with file and texture data to append to our output file as a new row
    List = [image_name,str(ts)[1:-1],str(us)[1:-1],str(vs)[1:-1],str(ws)[1:-1],str(xs)[1:-1],str(ys)[1:-1],str(zs)[1:-1],int(roi[1]),int(roi[3]),int(roi[0]),int(roi[2]),dr,depth,nodule,mineral,comment] 
    cv2.destroyWindow('ROI selector')


    # Open our existing CSV file in append mode
    # Create a file object for this file
    
    #with open('event.csv', 'a') as f_object:
    with open('output.csv', 'a') as f_object:

        
        # Pass this file object to csv.writer()
        # and get a writer object
        writer_object = writer(f_object)
 
        # Pass the list as an argument into
        # the writerow()
        writer_object.writerow(List)
 
        # Close the file object
        f_object.close()

The cell below runs the function "crop_save_texture" on the content of the irectory 'images'.  




The data output from the texture analysis was examined in a Principal Component Analysis (PCA).  The PCA wa run in the statistical computing environment R. 

In [ ]:
### get a list of file names in the directory "images" so that the code can run  in a loop on each image in the list
dirs=os.listdir(home+'/images/')

###Run the function "crop_save_texture" defined above.  This will append data to the output.csv file and save a  croped image of the region of interest. 
for file in dirs:
    print(home+'/images/'+file)
    print("Use the mouse to select an ROI, then press enter or return to save the ROI")
    image=cv2.imread(home+'/images/'+file,cv2.IMREAD_GRAYSCALE)[:,]
    crop_save_texture(image, file)


In [ ]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p wget,pandas,numpy,geopy,altair,vega,vega_datasets,watermark 

# date
print (" ")
%watermark -u -n -t -z 

In [3]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p cv2,csv,os,skimage,watermark 

# date
print (" ")
%watermark -u -n -t -z 

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.6.13
IPython 7.16.1

cv2 3.4.2
csv 1.0
os unknown
skimage 0.17.2
watermark 2.0.2

compiler   : GCC 7.5.0
system     : Linux
release    : 3.10.0-1160.83.1.el7.x86_64
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
 
last updated: Sun Apr 09 2023 12:27:30 CEST
